# **Machine Learning Project**

In questa esercitazione metteremo assieme tutte le nozioni apprese dall'inizio del corso per risolvere un task specifico di machine learning.

### **Task: Semi-supervised classification**

Il task che vogliamo risolvere è un task di classificazione, caratterizzato però dal fatto che solo una piccola parte dei dati che disponiamo possiede le annotazioni (label). Questa condizione è nota come `Semi-supervised learning`.

### **Dataset**

Il dataset che utilizzeremo sarà Fashion-MNIST, che contiene immagini di articoli di Zalando, composto da un training set di 60.000 campioni e test set con 10.000 campioni. Ogni campione è in scala di grigi e ha risoluzione 28x28. Il dataset è composto da 10 classi.


## **Pseudo-label**

Lo **pseudo-labeling** è una tecnica utilizzata nell'ambito del *semi-supervised learning*. L'idea di base è quella di generare etichette "artificiali" (pseudo-etichette) per i dati non etichettati (unlabeled, "U"), in modo da utilizzarle durante il training del modello. Per generare queste etichette ci sono diverse strategie: nel contesto di questa esercitazione utilizzeremo un algoritmo di clustering (k-means).

Ecco i passaggi generali del processo di pseudo-labeling:

1.  **Addestramento Iniziale**: Si addestra un algortimo di clustering sul set non etichettato (U) utilizzando un numero di cluster pari al numero di classi.
2.  **Predizione su Dati Etichettati**: Utilizziamo l'algortimo addestrato al punto 1 per clusterizzare i dati etichettati (L), assegnandoli quindi ai cluster che abbiamo trovato durante l' addestramento iniziale.
3.  **Mappare i cluster alle etichette**: Creiamo un mapping tra i cluster e le etichette, in modo da capire quale etichetta corrisponde allo specifico cluster. Per fare ciò assegniamo ad ogni cluster la vera label più frequente assegnata a quel cluster, sfruttando la funzione `mode`:

```Python
from scipy.stats import mode
import numpy as np

etichette_nel_cluster = np.array([0, 1, 1, 2, 1, 0, 1])

risultato_mode = mode(etichette_nel_cluster)

print(f"Oggetto ModeResult: {risultato_mode}") # Output: ModeResul(mode=1, count=4)
print(f"Etichetta più frequente (moda): {risultato_mode.mode}") # Output: (moda): 1

# etichetta più frequente come singolo numero:
etichetta_predominante = risultato_mode.mode
print(f"Etichetta predominante per questo cluster: {etichetta_predominante}") # Output: 1
```

4.  **Estrazione pseudo-label**: Alla fine, la classe più presente in un cluster diventa l' etichetta scelta per tutti i campioni assegnati a quel cluster.


**Vantaggi**:
*   Permette di sfruttare la grande quantità di dati non etichettati, che altrimenti andrebbero sprecati.
*   Può migliorare significativamente le prestazioni del modello rispetto all'addestramento con i soli dati etichettati, specialmente quando questi ultimi sono scarsi.

In [108]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode # For majority voting
from sklearn.neural_network import MLPClassifier

In [109]:
# Nomi delle classi per Fashion-MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### `load_and_preprocess_data()`

In questa funzione dovrete:

* Scaricare il dataset.
* Riordinare casualmente i dati.
* Effettuare reshape.
* Scalare i valori dei pixel all' intervallo [0,1].
* Ridurre il numero di campioni a 10.000 per il train e 1.000 per il test.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set ridotto.
2. Le etichette di train ridotte.
3. Il test set ridotto.
4. Le etichette di test ridotte.

In [110]:
def load_and_preprocess_data():
    """Carica e pre-processa il dataset Fashion-MNIST."""
    np.random.seed(0)

    # Carica il dataset
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

    # Riordina casualmente i dati
    train_indices = np.random.permutation(len(x_train))
    test_indices = np.random.permutation(len(x_test))

    x_train = x_train[train_indices]
    y_train = y_train[train_indices]
    x_test = x_test[test_indices]
    y_test = y_test[test_indices]

    # Riduci il numero di campioni
    x_train = x_train[:10000]
    y_train = y_train[:10000]
    x_test = x_test[:1000]
    y_test = y_test[:1000]

    # Reshape e normalizzazione
    x_train = x_train.reshape(-1, 28*28) / 255.0
    x_test = x_test.reshape(-1, 28*28) / 255.0

    print(f"Shape x_train: {x_train.shape}")
    print(f"Shape y_train: {y_train.shape}")
    print(f"Shape x_test: {x_test.shape}")
    print(f"Shape y_test: {y_test.shape}")

    return x_train, y_train, x_test, y_test

### `apply_pca_and_scale`

In questa funzione dovrete:

* Scalare il training set e il test set.
* Applicare PCA con un numero di componenti specificato come parametro della funzione (o, equivalentemente, con una frazione desiderata della varianza espressa).
* Stampare il numero di componenti.
* Stampare la varianza espressa.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set trasformato con PCA.
2. Il test set trasformato con PCA.

In [111]:
def apply_pca_and_scale(x_train, x_test, n_components):
    """Applica StandardScaler e PCA."""
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    pca = PCA(n_components=n_components)
    x_train_pca = pca.fit_transform(x_train_scaled)
    x_test_pca = pca.transform(x_test_scaled)

    print(f"PCA applicata. Numero di componenti selezionate: {pca.n_components_}")
    print(f"Varianza totale spiegata: {np.sum(pca.explained_variance_ratio_):.4f}")

    print(f"Shape x_train_pca: {x_train_pca.shape}")
    print(f"Shape x_test_pca: {x_test_pca.shape}")

    return x_train_pca, x_test_pca

### `create_semi_supervised_split`

In questa funzione dovrete:

* Splittare il training set in due insiemi, etichettato (L) e non etichettato (U) utilizzando  `train_test_split` con:

`test_size`=`(1.0 - labeled_fraction)`

* Stampare la shape del set etichettato.
* Stampare la shape del set non etichettato.

La funzione deve ritornare nell seguente ordine:

1. Il set etichettato.
2. Le etichette del set etichettato.
3. Il set non etichettato.
4. Le etichette del set non etichettato. **N.B.** Queste etichette verranno utilizzate **SOLO** per valutare le pseudo-labels, non per l'addestramento.


In [112]:
def create_semi_supervised_split(x_train_pca, y_train, labeled_fraction):
    """Crea gli insiemi etichettato (L) e non etichettato (U)."""
    x_labeled, x_unlabeled, y_labeled, y_unlabeled = train_test_split(
        x_train_pca, y_train, test_size=1.0-labeled_fraction, random_state=42)

    print(f"Dimensione insieme etichettato (L): {len(y_labeled)}")
    print(f"Dimensione insieme non etichettato (U): {len(y_unlabeled)}")

    return x_labeled, y_labeled, x_unlabeled, y_unlabeled

### `get_pseudo_labels`

In questa funzione dovrete:

* Istanziare un algoritmo di clustering (ad esempio, k-means).
* Addestrare e predire i clustering sul set non etichettato.
* Predire i clustering del set etichettato.
* Mappare i cluster ad un etichetta, utilizzando per ogni cluster l'etichetta più presente, estraibile utilizzando la funzione `mode` presentata sopra.
* Generare un array `pseudo_labels` assegnando a ogni campione del set non etichettato l'etichetta corrispondente al cluster a cui è stato assegnato.

La funzione deve ritornare:

1. L' array `pseudo_labels`.

In [113]:
def get_pseudo_labels(x_unlabeled_pca, x_labeled_pca, y_labeled, n_clusters):
    # 1. Istanziare algoritmo di clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)

    # 2. Allenare il modello di clustering con i dati non etichettati e salvare le assegnazioni ai cluster
    cluster_assignments_unlabeled = kmeans.fit_predict(x_unlabeled_pca)

    # 3. Calcolare l'assegnamento dei dati etichettati ai cluster
    cluster_assignments_labeled = kmeans.predict(x_labeled_pca)

    # 4. Mappiamo i cluster alle label vere più frequenti
    cluster_to_true_label_map = {}
    for k_idx in range(n_clusters):
        # 4.1 Troviamo per ogni cluster le etichette vere dei campioni che vi appartengono.
        labels_in_cluster = y_labeled[cluster_assignments_labeled == k_idx]

        if len(labels_in_cluster) > 0:
            # 4.2 Troviamo l'etichetta più frequente per quel cluster.
            mode_result = mode(labels_in_cluster)
            # Correzione qui: prendiamo solo il primo valore (mode può restituire array)
            cluster_to_true_label_map[k_idx] = mode_result.mode[0] if isinstance(mode_result.mode, np.ndarray) else mode_result.mode
        else:
            print(f"Attenzione: Cluster {k_idx} non ha campioni etichettati per il mapping. Assegno NaN.")
            cluster_to_true_label_map[k_idx] = np.nan

    pseudo_labels_list = []

    for c_assign in cluster_assignments_unlabeled:
        if c_assign in cluster_to_true_label_map:
            pseudo_labels_list.append(cluster_to_true_label_map[c_assign])
        else:
            # Se il cluster non ha una mappatura, possiamo assegnare un'etichetta di default o ignorarlo
            pseudo_labels_list.append(np.nan)

    # 5. Convertiamo la lista in un array numpy
    pseudo_labels = np.array(pseudo_labels_list)
    print(pseudo_labels.shape)

    return pseudo_labels

### `train_and_evaluate_classifier`

In questa funzione dovrete:

* Rimuovere eventuali campioni con etichette NaN (potrebbero provenire da pseudo labels non mappate).
* Istanziare il modello utilizzando `model_class` come oggetto e `classifier_args` come argomenti. Esempio:

```Python
model_class = MLPClassifier
classifier_args = {'max_iter': 200, 'hidden_layer_sizes': (100, 50)}
model = model_class(**classifier_args)

# Equivalente a:
model = MLPClassifier(max_iter=200, hidden_layer_sizes=(100, 50))

```

* Allenare il modello sul training set a cui sono stati rimossi i campioni con etichette NaN.
* Calcolare l' accuracy.
* Stampare il `title`, che consiste nel titolo dell' esperimento eseguito. Questo perchè tale funzione verrà riutilizzata diverse volte per più set di dati. Un titolo ci permetterà di identificare quali risultati stiamo producendo.
* Stampare l' accuracy.
* Stampare il classification report.

La funzione deve ritornare:

1. Il modello.


In [114]:
def train_and_evaluate_classifier(model_class, classifier_args, x_train, y_train, x_test, y_test, title, class_names_list):
    valid_indices_train = ~np.isnan(y_train)
    x_train_filtered = x_train[valid_indices_train]
    y_train_filtered = y_train[valid_indices_train]

    model = model_class(**classifier_args)
    model.fit(x_train_filtered, y_train_filtered)

    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"\n{title}")
    print(f"Accuratezza su test set: {accuracy:.4f}")
    print(classification_report(y_test, y_pred, target_names=class_names_list))

    return model

### `main`

In questa funzione dovrete:

* Utilizzare la funzione `load_and_preprocess_data` per caricare e pre-processare i dati.
* Utilizzare la funzione `apply_pca_and_scale` per applicare scaling e PCA.
* Utilizzare la funzione `create_semi_supervised_split` per dividere il train set in set etichettato e non etichettato.
* Utilizzare la funzione `get_pseudo_labels` per calcoalre le pseudo labels sul set non etichettato.
* Calcolare l' accuracy delle pseudo labels, cioè confrontarle con quelle vere in modo da vedere quanto sono accurate.
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati etichettati (L).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello sui dati etichettati (L) più quelli non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello su tutto il dataset originale.

In [115]:
def main(classifier_class, classifier_args, n_components_pca, labeled_fraction, n_clusters):
    # 1. Carica e pre-processa i dati
    try:
        x_train, y_train, x_test, y_test = load_and_preprocess_data()
        print(f"Dimensioni training set: {x_train.shape}, {y_train.shape}")
        print(f"Dimensioni test set: {x_test.shape}, {y_test.shape}")
    except Exception as e:
        print(f"Errore nel caricamento dati: {str(e)}")
        return

    # 2. Applica PCA e scaling
    try:
        x_train_pca, x_test_pca = apply_pca_and_scale(x_train, x_test, n_components_pca)
    except Exception as e:
        print(f"Errore in PCA: {str(e)}")
        return

    # 3. Crea split semi-supervised
    try:
        x_labeled_pca, y_labeled, x_unlabeled_pca, y_unlabeled = create_semi_supervised_split(
            x_train_pca, y_train, labeled_fraction)
    except Exception as e:
        print(f"Errore nella creazione dello split: {str(e)}")
        return

    # 4. Ottieni pseudo-labels
    try:
        pseudo_labels = get_pseudo_labels(x_unlabeled_pca, x_labeled_pca, y_labeled, n_clusters)

        # Calcola accuratezza pseudo-labels solo su campioni validi
        valid_pseudo_indices = ~np.isnan(pseudo_labels)
        if sum(valid_pseudo_indices) > 0:  # Verifica che ci siano campioni validi
            pseudo_accuracy = accuracy_score(y_unlabeled[valid_pseudo_indices],
                                          pseudo_labels[valid_pseudo_indices])
            print(f"\nAccuratezza delle pseudo-etichette (sui campioni mappabili di U): {pseudo_accuracy:.4f}")
        else:
            print("\nNessuna pseudo-label valida generata!")
            return
    except Exception as e:
        print(f"Errore nella generazione pseudo-labels: {str(e)}")
        return

    # 5. Valutazione modelli
    try:
        # 5.1 Baseline - Solo dati etichettati (L)
        print("\n" + "="*50)
        model_l = train_and_evaluate_classifier(
            classifier_class, classifier_args,
            x_labeled_pca, y_labeled,
            x_test_pca, y_test,
            "Baseline - Solo dati etichettati (L)",
            class_names
        )

        # 5.2 Solo dati con pseudo-etichette (U_pseudo)
        print("\n" + "="*50)
        model_u = train_and_evaluate_classifier(
            classifier_class, classifier_args,
            x_unlabeled_pca, pseudo_labels,
            x_test_pca, y_test,
            "Solo dati con pseudo-etichette (U_pseudo)",
            class_names
        )

        # 5.3 Combinato - Dati etichettati (L) + Pseudo-etichette (U_pseudo)
        print("\n" + "="*50)
        x_combined = np.concatenate([x_labeled_pca, x_unlabeled_pca[valid_pseudo_indices]])
        y_combined = np.concatenate([y_labeled, pseudo_labels[valid_pseudo_indices]])

        model_combined = train_and_evaluate_classifier(
            classifier_class, classifier_args,
            x_combined, y_combined,
            x_test_pca, y_test,
            "Combinato - Dati etichettati (L) + Pseudo-etichette (U_pseudo)",
            class_names
        )

        # 5.4 Oracle - Supervisione completa (intero training set)
        print("\n" + "="*50)
        model_oracle = train_and_evaluate_classifier(
            classifier_class, classifier_args,
            x_train_pca, y_train,
            x_test_pca, y_test,
            "Oracle - Supervisione completa (intero training set)",
            class_names
        )

        return model_l, model_u, model_combined, model_oracle

    except Exception as e:
        print(f"Errore nell'addestramento modelli: {str(e)}")
        return None

### **Utilizzare la funzione `main`**

Specifichiamo adesso un set di parametri richiesti dalla funzione main e utilizziamola. Nello specifico la funzione main ha bisogno di:

* `classifier_class`: quale classificatore utilizzare, ad esempio `'MLPClassifier'`, `'LogisticRegression'` o altri visti in precedenza.
* `classifier_args`: un dizionario contenente i parametri del classificatore scelto, ad esempio un `MLPClassifier` necessiterà del parametro `hidden_layer_sizes`. Dipendentemente da quale classificatore scegliete dovrete creare il dizionario.
* `n_components_pca`: numero di componenti di PCA che vogliamo utilizzare. Se specifichiamo un valore compreso in [0, 1] questo verrà considerato come la percentuale di varianza che vogliamo mentenere.
* `labeled_fraction`: percentuale di dati da usare come insieme etichettato. Si consiglia il valore 0.002 corrispondente allo 0.2%, cioè 16 immagini su 8000.
* `n_clusters`: numero di cluster da utilizzare, nel nostro caso vogliamo che ci sia un cluster per ogni classe, quindi 10.

Infine utilizziamo la funzione main.

In [116]:
# Parametri
CLASSIFIER_CLASS = MLPClassifier  # Modello da usare, ad esempio LogisticRegression o SVC
CLASSIFIER_ARGS = {
    'max_iter': 20,
    'hidden_layer_sizes': (200,200)  # Aumenta il numero di iterazioni per la convergenza
}
N_COMPONENTS_PCA = 0.95  # Mantiene il 95% della varianza spiegata, o un numero fisso es. 50
LABELED_FRACTION = 0.002   # Frazione di dati da usare come insieme etichettato L
N_CLUSTERS = 10          # Fashion-MNIST ha 10 classi

In [117]:
main(
    CLASSIFIER_CLASS,
    CLASSIFIER_ARGS,
    N_COMPONENTS_PCA,
    LABELED_FRACTION,
    N_CLUSTERS
)

Shape x_train: (10000, 784)
Shape y_train: (10000,)
Shape x_test: (1000, 784)
Shape y_test: (1000,)
Dimensioni training set: (10000, 784), (10000,)
Dimensioni test set: (1000, 784), (1000,)
PCA applicata. Numero di componenti selezionate: 245
Varianza totale spiegata: 0.9502
Shape x_train_pca: (10000, 245)
Shape x_test_pca: (1000, 245)
Dimensione insieme etichettato (L): 20
Dimensione insieme non etichettato (U): 9980
(9980,)

Accuratezza delle pseudo-etichette (sui campioni mappabili di U): 0.4379



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(



Baseline - Solo dati etichettati (L)
Accuratezza su test set: 0.4350
              precision    recall  f1-score   support

 T-shirt/top       0.61      0.40      0.48        85
     Trouser       0.97      0.37      0.54        91
    Pullover       0.55      0.15      0.23       109
       Dress       0.43      0.79      0.56       102
        Coat       0.40      0.71      0.52       112
      Sandal       0.25      0.91      0.40       104
       Shirt       0.33      0.03      0.06        95
     Sneaker       0.86      0.19      0.31       101
         Bag       0.93      0.12      0.22       113
  Ankle boot       0.78      0.67      0.72        88

    accuracy                           0.43      1000
   macro avg       0.61      0.44      0.40      1000
weighted avg       0.61      0.43      0.40      1000




/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and bein


Solo dati con pseudo-etichette (U_pseudo)
Accuratezza su test set: 0.4070
              precision    recall  f1-score   support

 T-shirt/top       0.35      0.38      0.36        85
     Trouser       0.66      0.87      0.75        91
    Pullover       0.00      0.00      0.00       109
       Dress       0.39      0.46      0.42       102
        Coat       0.43      0.58      0.49       112
      Sandal       0.29      0.84      0.43       104
       Shirt       0.14      0.15      0.14        95
     Sneaker       0.00      0.00      0.00       101
         Bag       0.00      0.00      0.00       113
  Ankle boot       0.72      0.94      0.82        88

    accuracy                           0.41      1000
   macro avg       0.30      0.42      0.34      1000
weighted avg       0.28      0.41      0.33      1000




/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and bein


Combinato - Dati etichettati (L) + Pseudo-etichette (U_pseudo)
Accuratezza su test set: 0.4020
              precision    recall  f1-score   support

 T-shirt/top       0.36      0.39      0.37        85
     Trouser       0.65      0.87      0.75        91
    Pullover       0.00      0.00      0.00       109
       Dress       0.36      0.44      0.40       102
        Coat       0.43      0.56      0.49       112
      Sandal       0.29      0.84      0.43       104
       Shirt       0.13      0.14      0.13        95
     Sneaker       0.00      0.00      0.00       101
         Bag       0.00      0.00      0.00       113
  Ankle boot       0.72      0.93      0.81        88

    accuracy                           0.40      1000
   macro avg       0.29      0.42      0.34      1000
weighted avg       0.28      0.40      0.32      1000



Oracle - Supervisione completa (intero training set)
Accuratezza su test set: 0.8570
              precision    recall  f1-score   support

 T-

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


(MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=20),
 MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=20),
 MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=20),
 MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=20))